In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
from sklearn import linear_model,datasets
from sklearn.metrics import accuracy_score
import scipy.stats as scipy
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier 
from sklearn import preprocessing
from sklearn.metrics import metrics
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import sklearn.grid_search
from sklearn.cluster import KMeans
from scipy.stats import randint as sp_randint
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
train_set1=pd.read_csv("Attributes11.csv")
train_set1.info()
train_set2=pd.read_csv("publicMeetingFixed.csv")
train_set2.info()
train_set3=pd.read_csv("regionCodefixed.csv")
train_set3.info()
train_set4=pd.read_csv("status_group.csv")
train_set4.info()
train_set=pd.merge(train_set4, train_set1, on='id')
train_set.head()

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\metrics.py:4: DeprecationWarning: sklearn.metrics.metrics is deprecated and will be removed in 0.18. Please import from sklearn.metrics
  DeprecationWarning)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,14-03-2011,Roman,1390,Roman,34.938093,-9.856322,none,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,06-03-2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,25-02-2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,28-01-2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,13-07-2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [2]:
#replace public_meeting with publicmeeting grouped by region
merge1=pd.merge(train_set, train_set2, on='region')
merge1.info()
#merge1["public_meeting_x"]=merge1.public_meeting_x.replace(r'\s+',np.nan,regex=True).replace('',np.nan)
#merge1.info()
print(merge1['public_meeting_x'].isnull().sum())
merge1['public_meeting_y'].fillna('True',inplace=True)
merge1.loc[merge1['public_meeting_x'].isnull(),'public_meeting_x'] = merge1['public_meeting_y']
merge1['public_meeting_x'].value_counts()
merge1['public_meeting']=merge1['public_meeting_x']
merge1.drop('public_meeting_y',axis=1,inplace=True)
merge1.drop('public_meeting_x',axis=1,inplace=True)
merge1.info()
merged_data=pd.merge(merge1, train_set3, on='region')
merged_data.info()
merged_data=pd.merge(train_set, train_set3, on='region')
merged_data['region_code_x']=merged_data['region_code_y']
merged_data['region_code'] = merged_data['region_code_x']
merged_data.drop('region_code_y',axis=1,inplace=True)
merged_data.drop('region_code_x',axis=1,inplace=True)
merged_data['construction_year'].replace(to_replace=0, value=merged_data['construction_year'].median(),inplace=True)
merged_data['construction_year'] = pd.cut(merged_data.construction_year, bins=20,labels=False)
merged_data['construction_year'].value_counts()
#setting lat and long with median based on lga -- works
merged_data.longitude.replace(0.000000,np.nan,inplace=True)
merged_data.latitude.replace(-2.000000e-08,np.nan,inplace=True)
merged_data['long_median'] = merged_data.groupby('lga')['longitude'].transform('median')
merged_data['lat_median'] = merged_data.groupby('lga')['latitude'].transform('median')
#merged_data
print(np.isnan(merged_data.longitude).sum())
merged_data.loc[np.isnan(merged_data.longitude),'longitude'] = merged_data['long_median']
merged_data.loc[np.isnan(merged_data.latitude),'latitude'] = merged_data['lat_median']
merged_data.info()
merged_data['long1_median'] = merged_data.groupby('district_code')['longitude'].transform('median')
merged_data['lat1_median'] = merged_data.groupby('district_code')['latitude'].transform('median')
merged_data.loc[np.isnan(merged_data.longitude),'longitude'] = merged_data['long1_median']
merged_data.loc[np.isnan(merged_data.latitude),'latitude'] = merged_data['lat1_median']
# testing
merged_data['date_recorded']=pd.to_datetime(merged_data['date_recorded'],format="%d-%m-%Y")
merged_data['date_recorded']=merged_data['date_recorded'] - pd.datetime(1970,1,1)
merged_data['date_recorded'] =merged_data['date_recorded'].astype('timedelta64[D]')
merged_data.info()
merged_data['date_recorded']=pd.cut(merged_data.date_recorded, bins=20,labels=False)
merged_data['funder'].value_counts()
merged_data['funder_freq']=merged_data.groupby('funder')['funder'].transform('count')
merged_data.ix[merged_data.funder_freq < 50, 'funder'] = 'others'
merged_data['funder'].value_counts()
merged_data['installer'].value_counts()
merged_data['installer_freq']=merged_data.groupby('installer')['installer'].transform('count')
merged_data.ix[merged_data.installer_freq < 50, 'installer'] = 'others'
merged_data.ix[merged_data.installer =='0', 'installer'] = 'others'
merged_data['installer'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 42 columns):
id                       59400 non-null int64
status_group             59400 non-null object
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
p

DWE                     17402
others                  10080
missing                  3655
Government               1825
RWE                      1206
Commu                    1060
DANIDA                   1050
KKKT                      898
Hesawa                    840
TCRS                      707
Central government        622
CES                       610
Community                 553
DANID                     552
District Council          551
HESAWA                    539
LGA                       408
World vision              408
WEDECO                    397
TASAF                     396
District council          392
Gover                     383
AMREF                     329
TWESA                     316
WU                        301
Dmdd                      287
ACRA                      278
World Vision              270
SEMA                      249
DW                        246
                        ...  
GOVERNMENT                 66
Villa                      65
MTN       

In [3]:
merged_data.drop('wpt_name',axis=1,inplace=True)
merged_data.drop('num_private',axis=1,inplace=True)
#merged_data.drop('scheme_management',axis=1,inplace=True)
#merged_data.drop('quantity_group',axis=1,inplace=True)
#merged_data.drop('source_type',axis=1,inplace=True)
#merged_data.drop('waterpoint_type_group',axis=1,inplace=True)
#merged_data.drop('payment_type',axis=1,inplace=True)
merged_data.drop('long_median',axis=1,inplace=True)
merged_data.drop('lat_median',axis=1,inplace=True)
merged_data.drop('long1_median',axis=1,inplace=True)
merged_data.drop('lat1_median',axis=1,inplace=True)
merged_data.drop('id',axis=1,inplace=True)
merged_data.drop('amount_tsh',axis=1,inplace=True)
#merged_data.drop('extraction_type_class',axis=1,inplace=True)
merged_data.drop('installer_freq',axis=1,inplace=True)
merged_data.drop('funder_freq',axis=1,inplace=True)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 37 columns):
status_group             59400 non-null object
date_recorded            59400 non-null int64
funder                   59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
basin                    59400 non-null object
subvillage               59400 non-null object
region                   59400 non-null object
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
recorded_by              59400 non-null object
scheme_management        59400 non-null object
scheme_name              59400 non-null object
permit                   59400 non-null object


In [4]:
columnsToEncode = list(merged_data.select_dtypes(include=['category','object']))
le = preprocessing.LabelEncoder()
merged_data=merged_data.fillna(0)
merged_data["funder"]=merged_data["funder"].astype(str)
merged_data["installer"]=merged_data["installer"].astype(str)
merged_data["subvillage"]=merged_data["subvillage"].astype(str)
merged_data["scheme_name"]=merged_data["scheme_name"].astype(str)
#merged_data['date_recorded']=merged_data['date_recorded'].astype(str)
#print (merged_data.funder.value_counts())
#merged_data["feature"] = le.fit_transform(merged_data["funder"])
for feature in columnsToEncode:
    try:
         merged_data[feature] = le.fit_transform(merged_data[feature])
    except:
         print('Error encoding '+feature)


In [5]:

data=merged_data.ix[ :, merged_data.columns != 'status_group']
target1=merged_data['status_group']
train1,test1,train2,test2=train_test_split(data,target1,test_size=0.2,random_state=0)
train1.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 55424 to 2732
Data columns (total 36 columns):
date_recorded            47520 non-null int64
funder                   47520 non-null int64
gps_height               47520 non-null int64
installer                47520 non-null int64
longitude                47520 non-null float64
latitude                 47520 non-null float64
basin                    47520 non-null int64
subvillage               47520 non-null int64
region                   47520 non-null int64
district_code            47520 non-null int64
lga                      47520 non-null int64
ward                     47520 non-null int64
population               47520 non-null int64
public_meeting           47520 non-null int64
recorded_by              47520 non-null int64
scheme_management        47520 non-null int64
scheme_name              47520 non-null int64
permit                   47520 non-null int64
construction_year        47520 non-null int64
extraction

In [20]:
#RANDOM FORESTS test train split for scoring

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 150)
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train1,train2)

# Take the same decision trees and run it on the test data
predicted_classification = forest.predict(test1)
print(forest.score(test1,test2))
print(forest.feature_importances_)
print(accuracy_score(test2, predicted_classification))

0.801515151515
[ 0.00790618  0.03245131  0.05615581  0.02451698  0.10623751  0.1037262
  0.01221134  0.07158959  0.01503993  0.01729218  0.02465935  0.04562986
  0.03815836  0.00650914  0.          0.01288338  0.02750531  0.00804376
  0.04027018  0.02940191  0.02879877  0.01440741  0.00632525  0.03148239
  0.01141214  0.01131676  0.12001635  0.01972838  0.00682973  0.05248594
  0.0170086 ]
0.801515151515


In [36]:
clf = Pipeline([('pca', PCA(n_components=36)),
                ('classification', RandomForestClassifier(n_estimators = 150))])
clf=clf.fit(train1, train2)
output =clf.predict(test1)
print(clf.score(test1,test2))

0.780976430976


In [38]:
#RANDOM FORESTS test train split for scoring

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 250)
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train1,train2)

# Take the same decision trees and run it on the test data
predicted_classification = forest.predict(test1)
print(accuracy_score(test2, predicted_classification))

0.804208754209


In [13]:
#RANDOM FORESTS test train split for scoring

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 500,criterion='entropy')
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train1,train2)

# Take the same decision trees and run it on the test data
predicted_classification = forest.predict(test1)
print(accuracy_score(test2, predicted_classification))
#250 estimators- 0.801936026936
#500 estimators- 0.801936026936

0.803451178451


In [39]:
#RANDOM FORESTS test train split for scoring

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 1500,criterion='entropy')
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train1,train2)

# Take the same decision trees and run it on the test data
predicted_classification = forest.predict(test1)
print(accuracy_score(test2, predicted_classification))
#150 estimators- 0.801599326599


0.805555555556


In [40]:
clf = GradientBoostingClassifier(n_estimators=250,learning_rate=1.0).fit(train1,train2)
predicted_classification = clf.predict(test1)
print(accuracy_score(test2, predicted_classification))
#100 - maxdepth-1, random_state=0 -0.79
#150 -0.778535353535
#250 - 0.76531986532 - without learning rate
#250 - learning rate=1 -0.780555555556
#250 -learning rate 2.0 - 0.698063973064

0.775252525253


In [27]:
#GRID search 
clf_forest2 = sklearn.ensemble.RandomForestClassifier()
n_iter_search = 10
param_dist = {"max_depth": [50],
"max_features": sp_randint(29, 31),
"min_samples_split": sp_randint(20, 22),
"min_samples_leaf": sp_randint(50, 51),
"criterion": [ "entropy"]}

random_search = sklearn.grid_search.RandomizedSearchCV(clf_forest2,
param_distributions = param_dist,
n_iter = n_iter_search)

random_search.fit(train1, train2)
print(random_search.score(train1, train2))
print(random_search.score(test1, test2))

0.789078282828
0.765572390572


In [1]:
kmeans = KMeans(n_clusters=3,init='k-means++', random_state=0).fit(train1)
output=kmeans.predict(test1)
print(kmeans.score(test1,test2))
print(kmeans.cluster_centers_)
print(metrics.silhouette_score(data, output,
                                      metric='euclidean'))

NameError: name 'KMeans' is not defined

In [ ]:
# for k-means
colormap = np.array(['red', 'lime', 'black'])
 
# Plot the Original Classifications
plt.subplot(1, 2, 1)
plt.scatter(x.Petal_Length, x.Petal_Width, c=colormap[y.Targets], s=40)
plt.title('Real Classification')
 
# Plot the Models Classifications
plt.subplot(1, 2, 2)
plt.scatter(x.Petal_Length, x.Petal_Width, c=colormap[model.labels_], s=40)
plt.title('K Mean Classification')

In [48]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
for i in range(3):
    # select only data observations with cluster label == i
    ds = output[np.where(labels==i)]
    # plot the data observations
    plt.plot(ds[:,0],ds[:,1],'o')
    # plot the centroids
    lines = plt.plot(centroids[i,0],centroids[i,1],'kx')
    # make the centroid x's bigger
    plt.setp(lines,ms=15.0)
    plt.setp(lines,mew=2.0)
plt.show()

IndexError: index 11881 is out of bounds for axis 1 with size 11880

In [ ]:
clf = SVC(max_iter=10000,probability=True)
clf.fit(train1,train2)
predicted_classification=clf.predict(test1)
print(accuracy_score(test2, predicted_classification))


In [6]:
clf = SVC(max_iter=10000,kernel='poly',degree=2.0,C=0.0000000001)
clf.fit(train1,train2)
predicted_classification=clf.predict(test1)
print(accuracy_score(test2, predicted_classification))

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:224: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


0.40968013468


In [6]:
clf = SVC(max_iter=10000,kernel='rbf',C=0.0000000001)
clf.fit(train1,train2)
predicted_classification=clf.predict(test1)
print(accuracy_score(test2, predicted_classification))

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:224: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


0.533249158249


In [41]:
clf = SVC(max_iter=50000,kernel='rbf',C=0.0000000001)
clf.fit(train1,train2)
predicted_classification=clf.predict(test1)
print(accuracy_score(test2, predicted_classification))

0.54234006734


In [8]:
clf = SVC(max_iter=99000,kernel='rbf',C=0.0000000001)
clf.fit(train1,train2)
predicted_classification=clf.predict(test1)
print(accuracy_score(test2, predicted_classification))

0.54234006734


In [22]:
h=0.2
logreg = linear_model.LogisticRegression(C=1e5)
# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(train1, train2)
Z = logreg.predict(test1)
print(logreg.score(test1,test2))
#testdf=np.array(test2)
#.to_frame(name='status_group')
#test1.info()
#print(test2.size)
#plt.scatter(test1, testdf,  color='black')
#plt.plot(test1, Z, color='blue',linewidth=3)

#plt.xticks(())
#plt.yticks(())

#plt.show()

0.635016835017


In [14]:
output=OneVsRestClassifier(SVC(random_state=0)).fit(train1, train2).predict(test1)
print(accuracy_score(test2, output))

0.595202020202


In [21]:
output=OneVsRestClassifier(RandomForestClassifier(n_estimators = 150,criterion='entropy')).fit(train1, train2).predict(test1)
print(accuracy_score(test2, output))
#0.806144781145 - without few cols

0.803872053872


In [10]:
output=OneVsRestClassifier(RandomForestClassifier(n_estimators = 150,criterion='gini')).fit(train1, train2).predict(test1)
print(accuracy_score(test2, output))

0.804124579125


In [13]:
output=OneVsRestClassifier(RandomForestClassifier(n_estimators = 150,criterion='entropy')).fit(train1, train2).predict(test1)
print(accuracy_score(test2, output))

0.801936026936


In [34]:
gnb = GaussianNB()
output = gnb.fit(train1, train2).predict(test1)
print(accuracy_score(test2, output))

0.604966329966


In [21]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3,algorithm='auto')
#train1=train1+train2
output=neigh.fit(train1,train2).predict(test1)
print(accuracy_score(test2, output))

0.622306397306


In [23]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(train1,train2)
output=clf.predict(test1)
print(accuracy_score(test2, output))

0.387626262626


In [26]:
reg = linear_model.Ridge (alpha = .5,solver='sparse_cg')
clf.fit(train1,train2)
output=clf.predict(test1)
print(accuracy_score(test2, output))

0.390740740741
